In [ ]:
import os
import warnings
import copy
import tifffile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.collections import LineCollection
from matplotlib.colors import LinearSegmentedColormap
from scipy.spatial import ConvexHull, QhullError

import sys
sys.path.append("..")
from DiffusionAnalysis import DiffusionAnalysis
import obf_support

# warning suppression
from pandas.errors import SettingWithCopyWarning
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)
warnings.filterwarnings('ignore')

plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['font.family'] = 'arial'

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Common analysis parameter values
# [site_rad, inclusion_rad, circle_radii, blob_dist, min_time, split_len_thresh, max_time_lag, max_dt, fit_len_thresh, meanslidestd_thresh, slidstd_interval, meanpos_thresh, interval_meanpos, interval_dist]
analysis_parameter_vals = [0.2, 0.05, [0.05,0.10,0.20], 0.013, 30e-3, 5, 500e-6, 1e-3, 10, 0.02, 40, 0.025, 5, 50]
figures = [0,1]

Local functions

In [ ]:
color_min    = "#004ddd"
color_center = "black"
color_max    = "#db0000"
cmap = LinearSegmentedColormap.from_list(
    "cmap_name",
    [color_min, color_min, color_min, color_min, color_min, color_min, color_center, color_max, color_max, color_max, color_max, color_max, color_max]
)

def packing_coeff_analysis_locs(track_data, n=25, pack_lim=0, plot=True):
    state_ratios = []
    state_changes_per_ms = []
    pcs_all = []
    pcslenmax = 0
    plt.figure(figsize=(10,3))
    for idx, track in track_data.iterrows():
        pcs = []
        if len(track['x']) > n*4 and track['filter'] == 0:
            x_track = track['x']
            y_track = track['y']
            t_track = track['tim']
            for i_mid in np.arange(0, (n//2)):
                pcs.append(0)
            for i_mid in np.arange((n//2), len(x_track)-(n//2)):
                pc = pack_coeff(x_track, y_track, t_track, n, i_mid)
                pcs.append(pc)
            for i_mid in np.arange(len(x_track)-(n//2),len(x_track)):
                pcs.append(pcs[-1])
            pcs[:n//2] = [pcs[n//2] for i in range(n//2)]
            if plot:
                plt.plot(pcs)
            # time spent in confined state, using threshold
            # find changepoints above/below threshold, and sum time spent in each state
            pc_state = np.array(pcs)>pack_lim
            conf_time = 0
            num_state_changes = 0
            if pc_state[0]:
                start_time = t_track[0]
            for i in range(1, len(pc_state)):
                if pc_state[i] != pc_state[i-1]: # state change
                    if pc_state[i]: # entering confined state
                        start_time = t_track[i]
                    else: # exiting confined state
                        end_time = t_track[i]
                        conf_time += end_time - start_time
                        num_state_changes += 1
            if pc_state[-1]: # if track ends in confined state, add remaining time
                conf_time += t_track[-1] - start_time
                num_state_changes += 1
            conf_state_ratio = conf_time/(t_track[-1] - t_track[0])
            num_state_changes_per_ms = num_state_changes/((t_track[-1] - t_track[0])*1000)
            state_ratios.append(conf_state_ratio)
            state_changes_per_ms.append(num_state_changes_per_ms)
            if len(pcs) > pcslenmax:
                pcslenmax = len(pcs)
            pcs_all.append(pcs)
        else:
            tracklen = len(track['x'])
            trackfilt = track['filter']
            pcs_all.append(np.nan)
            state_ratios.append(np.nan)
            state_changes_per_ms.append(np.nan)
    if plot:
        plt.hlines(pack_lim, xmin=0, xmax=pcslenmax, color='red')
        plt.show()
    track_data['pc'] =  pcs_all
    track_data['state_ratios'] =  state_ratios
    track_data['state_changes_per_ms'] =  state_changes_per_ms
    return track_data

def pack_coeff(x, y, t, n, i_mid):
    """ Modified function from https://www.cell.com/biophysj/pdf/S0006-3495(17)31029-9.pdf, where I make a sqdist/time, instead of just sqdist, to take into account the uneven steps. """
    sqd_tims = []
    for i in np.arange(i_mid-n//2, i_mid+n//2):
        sqd = (x[i+1]-x[i])**2+(y[i+1]-y[i])**2
        #dt = t[i+1]-t[i]
        sqd_tims.append(sqd)#/dt)
    S = ConvexHull(np.swapaxes(np.vstack([x[i_mid-n//2:i_mid+n//2], y[i_mid-n//2:i_mid+n//2]]),0,1)).volume
    pc = np.sum(sqd_tims)/S**2
    return pc

def plot_traj_conf(track_data, savefolder, extend=1, seg_mode='both'):
    plot_roiidx = -1
    plot_confidx = -1
    last_lc = None
    for idx, track in track_data.iterrows():
        if track['filter'] == 0:
            curr_roiidx = track['roiidx']
            curr_confidx = track['confidx']
            if curr_roiidx != plot_roiidx or curr_confidx != plot_confidx:
                if plot_roiidx != -1 and plot_roiidx != -1:
                    plt.axis('off')
                    plt.savefig(os.path.join(savefolder, f'trajPack-{plot_confidx}-{plot_roiidx}.png'), format="png", bbox_inches="tight", dpi=600)
                    plt.show()
                plot_roiidx = curr_roiidx
                plot_confidx = curr_confidx
                fig, ax = plt.subplots(figsize=(1.6,1.6))
                confimgplot = plt.imshow(track['confimg'], cmap='gray', alpha=0.5)
                confimgplot.set_extent(track['confimg_ext'])
                plt.xlim(*np.array(track['conf_xlim'])+[-extend,extend])
                plt.ylim(*np.array(track['conf_ylim'])+[extend,-extend])
            if not isinstance(track['pc'], (list, np.ndarray)):
                continue
            
            # Need confinement mask
            mask = track.get('conf_mask', None)
            if mask is None:
                continue
            mask = np.asarray(mask, dtype=bool)

            x = np.asarray(track['x'])
            y = np.asarray(track['y'])
            if len(x) < 2 or len(mask) != len(x):
                continue
            col = mask_to_segment_values(mask, mode=seg_mode)  # length N-1
            line, lc = colored_line_between_pts(x, y, col, ax, cmap=cmap, linewidth=0.2, alpha=0.3)
            # Force binary color scaling
            lc.set_clim(0, 1)
            last_lc = lc
            
    if plot_roiidx != -1 and plot_confidx != -1:)
        plt.axis('off')
        plt.savefig(os.path.join(savefolder, f'trajPack-{plot_confidx}-{plot_roiidx}.png'), format="png", bbox_inches="tight", dpi=600)
        plt.show()

def mask_to_segment_values(mask, mode="both"):
    m = np.asarray(mask, dtype=bool)
    if m.size < 2:
        return np.array([], dtype=float)
    if mode == "both":
        seg = m[:-1] & m[1:]
    elif mode == "any":
        seg = m[:-1] | m[1:]
    else:
        raise ValueError("mode must be 'both' or 'any'")
    return seg.astype(float)  # 0/1

def colored_line_between_pts(x, y, c, ax, **lc_kwargs):
    """
    Plot a line with a color specified between (x, y) points by a third value.
    """
    if "array" in lc_kwargs:
        warnings.warn('The provided "array" keyword argument will be overridden')

    # Check color array size (LineCollection still works, but values are unused)
    if len(c) != len(x) - 1:
        warnings.warn(
            "The c argument should have a length one less than the length of x and y. "
            "If it has the same length, use the colored_line function instead."
        )

    # Create a set of line segments so that we can color them individually
    # This creates the points as an N x 1 x 2 array so that we can stack points
    # together easily to get the segments. The segments array for line collection
    # needs to be (numlines) x (points per line) x 2 (for x and y)
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    lc = LineCollection(segments, **lc_kwargs)
    # Set the values used for colormapping
    lc.set_array(c)

    return ax.add_collection(lc), lc

def has_pc(track):
    pc = track.get('pc', np.nan)
    return not (np.isscalar(pc) and np.isnan(pc))

def event_diameter_bbox(xe, ye):
    dx = xe.max() - xe.min()
    dy = ye.max() - ye.min()
    return np.sqrt(dx*dx + dy*dy)

def net_displacement(xe, ye):
    dx = xe[-1] - xe[0]
    dy = ye[-1] - ye[0]
    return np.sqrt(dx*dx + dy*dy)

def label_confinement_mask(track_data, t_win, pack_lim,
                           min_dur_s=0, min_pts=15,
                           confinement_size_thresh=0):
    """
    Returns a boolean mask (len = n_locs) marking confined localizations.
    Uses Pc>pack_lim candidate runs, then applies:
      - min duration (seconds)
      - min points
      - bbox diameter < confinement_size_thresh (µm)
      - net displacement < confinement_size_thresh/2 (µm)
    And shrinks each accepted event by t_win/2 to undo Pc smearing.
    """
    conf_masks_all = []
    conf_events_all = []
    for idx, track in track_data.iterrows():
        n = len(track['x'])
        mask = np.zeros(n, dtype=bool)
        if not has_pc(track):
            conf_masks_all.append(mask)
            conf_events_all.append(np.nan)
            continue
        pc = np.asarray(track['pc'])
        t = np.asarray(track['tim'])
        x = np.asarray(track['x'])
        y = np.asarray(track['y'])
        above = pc > pack_lim

        # find runs of True
        events = []
        i = 0
        while i < n:
            if not above[i]:
                i += 1
                continue
            j = i
            while j + 1 < n and above[j + 1]:
                j += 1

            # candidate run is [i, j]
            t0, t1 = t[i], t[j]
            dur = t1 - t0
            pts = j - i + 1
            if dur >= min_dur_s and pts >= min_pts:
                xe = x[i:j+1]
                ye = y[i:j+1]
                diam = event_diameter_bbox(xe, ye)
                netd = net_displacement(xe, ye)
                if diam <= confinement_size_thresh*3 and netd <= confinement_size_thresh*1.5:
                    #shrink by half window in time to avoid coloring outside
                    shrink = t_win / 2
                    t0c = t0 + shrink
                    t1c = t1 - shrink
                    if t1c > t0c:
                        # map core time interval back to indices
                        core_idx = np.where((t >= t0c) & (t <= t1c))[0]
                        if core_idx.size > 0:
                            mask[core_idx] = True
                            events.append((i, j, t0, t1, t0c, t1c, dur, diam))
            i = j + 1
        conf_masks_all.append(mask)
        conf_events_all.append(events)
    track_data['conf_mask'] = conf_masks_all
    track_data['conf_events'] = conf_events_all  # keep intervals for debugging/plot legends
    return track_data

def build_confinement_coords_from_mask(track_data):
    """
    From track['conf_mask'], build track['confinement_coords'] as:
      (x_mean, y_mean, t_mean, duration_seconds)
    using *contiguous True runs* in conf_mask.
    """
    coords_all = []
    for idx, track in track_data.iterrows():
        if 'conf_mask' not in track or track['conf_mask'] is None:
            coords_all.append(np.nan)
            continue

        m = np.asarray(track['conf_mask'], dtype=bool)
        x = np.asarray(track['x'])
        y = np.asarray(track['y'])
        t = np.asarray(track['tim'])

        if m.size == 0 or not np.any(m):
            coords_all.append(np.nan)
            continue

        coords = []
        i = 0
        n = len(m)
        while i < n:
            if not m[i]:
                i += 1
                continue
            j = i
            while j + 1 < n and m[j + 1]:
                j += 1
            xe, ye, te = x[i:j+1], y[i:j+1], t[i:j+1]
            dur = te[-1] - te[0]
            coords.append((float(np.mean(xe)), float(np.mean(ye)), float(np.mean(te)), float(dur)))
            i = j + 1
        coords_all.append(coords)
    track_data['confinement_coords'] = coords_all
    return track_data

def plot_confinements(track_data, savefolder, extend=1):
    plot_roiidx = -1
    plot_confidx = -1
    for idx, track in track_data.iterrows():
        if track['filter'] == 0:
            curr_roiidx = track['roiidx']
            curr_confidx = track['confidx']
            if curr_roiidx != plot_roiidx or curr_confidx != plot_confidx:
                if plot_roiidx != -1 and plot_confidx != -1:
                    plt.savefig(os.path.join(savefolder, f'confinements-{plot_confidx}-{plot_roiidx}.png'), format="png", bbox_inches="tight", dpi=250)
                    plt.show()
                plot_roiidx = curr_roiidx
                plot_confidx = curr_confidx
                fig, ax = plt.subplots(figsize=(5,5))
                confimgplot = plt.imshow(track['confimg'], cmap='gray', alpha=0.2)
                confimgplot.set_extent(track['confimg_ext'])
                plt.xlim(*np.array(track['conf_xlim'])+[-extend,extend])
                plt.ylim(*np.array(track['conf_ylim'])+[extend,-extend])
            if track['confinement_coords'] is not np.nan:
                plt.scatter([coord[0] for coord in track['confinement_coords']], [coord[1] for coord in track['confinement_coords']], s=[coord[3]*1000 for coord in track['confinement_coords']], c='red', alpha=0.3)
    plt.savefig(os.path.join(savefolder, f'confinements-{plot_confidx}-{plot_roiidx}.png'), format="png", bbox_inches="tight", dpi=250)
    plt.show()
    
    def radial_profile_confinements(trackdata, dr=0.05, rmax=None):
    r_loc_all = []
    r_conf_all = []

    for idx, track in trackdata.iterrows():
        x0, y0 = track["roipos"]
        # confinement centers
        conf_list = track["confinement_coords"]
        if isinstance(conf_list, (list, np.ndarray)):
            if conf_list is not None and len(conf_list) > 0:
                conf_arr = np.asarray(conf_list, dtype=float)  # shape (Nconf, 3) or (Nconf, 4)
                xc = conf_arr[:, 0]
                yc = conf_arr[:, 1]
                r_conf = np.sqrt((xc - x0)**2 + (yc - y0)**2)
                r_conf_all.append(r_conf)
                # localizations, only for tracks with confinements
                x = np.asarray(track["x"])
                y = np.asarray(track["y"])
                r_loc = np.sqrt((x - x0)**2 + (y - y0)**2)
                r_loc_all.append(r_loc)

    r_loc_all = np.concatenate(r_loc_all) if len(r_loc_all) else np.array([])
    r_conf_all = np.concatenate(r_conf_all) if len(r_conf_all) else np.array([])

    if rmax is None:
        rmax = max(r_loc_all.max(initial=0), r_conf_all.max(initial=0))

    bins = np.arange(0, rmax + dr, dr)
    L, _ = np.histogram(r_loc_all, bins=bins)   # localizations in tracks with confinements per annulus
    S, _ = np.histogram(r_conf_all, bins=bins)  # confinement events per annulus

    P = np.full_like(L, np.nan, dtype=float)
    mask = L > 0
    P[mask] = S[mask] / L[mask]

    r_centers = 0.5 * (bins[:-1] + bins[1:])
    return r_centers, P, S, L

def radial_profile_conf_fraction(trackdata, dr=0.05, rmax=None):
    r_loc_all = []
    dt_loc_all = []
    
    r_conf_all = []
    dt_conf_all = []

    for idx, track in trackdata.iterrows():
        x0, y0 = track["roipos"]

        # confinement events (xc, yc, tmean, dt_conf)
        conf_list = track["confinement_coords"]
        if isinstance(conf_list, (list, np.ndarray)):
            if conf_list is not None and len(conf_list) > 0:
                conf_arr = np.asarray(conf_list, dtype=float)
                xc = conf_arr[:, 0]
                yc = conf_arr[:, 1]
                dt_conf = conf_arr[:, 3]
                r_conf = np.sqrt((xc - x0)**2 + (yc - y0)**2)
                r_conf_all.append(r_conf)
                dt_conf_all.append(dt_conf)
                # localizations, only for tracks with confinements
                # localization radii
                x = np.asarray(track["x"])
                y = np.asarray(track["y"])
                t = np.asarray(track["tim"])
                r_loc = np.sqrt((x - x0)**2 + (y - y0)**2)
                # time step weights (same length as r_loc), append median dt to last to avoid zero effect
                dt = np.diff(t, append=t[-1] + np.median(np.diff(t)))
                r_loc_all.append(r_loc)
                dt_loc_all.append(dt)

    r_loc_all = np.concatenate(r_loc_all)
    dt_loc_all = np.concatenate(dt_loc_all)

    r_conf_all = np.concatenate(r_conf_all) if len(r_conf_all) else np.array([])
    dt_conf_all = np.concatenate(dt_conf_all) if len(dt_conf_all) else np.array([])

    if rmax is None:
        rmax = max(r_loc_all.max(initial=0), r_conf_all.max(initial=0))

    bins = np.arange(0, rmax + dr, dr)

    # denominator: total observed time per annulus
    Tobs, _ = np.histogram(r_loc_all, bins=bins, weights=dt_loc_all)

    # numerator: total confined time per annulus
    Tconf, _ = np.histogram(r_conf_all, bins=bins, weights=dt_conf_all)

    P = np.full_like(Tobs, np.nan, dtype=float)
    mask = Tobs > 0
    P[mask] = Tconf[mask] / Tobs[mask]

    r_centers = 0.5 * (bins[:-1] + bins[1:])
    return r_centers, P, Tconf, Tobs

def radial_conf_fraction_from_intervals(tracks, dr=0.05, rmax=None, min_time_per_bin=0.0, bins=None):
    """
    Fraction of time confined vs radius, computed from localization radii.
    Numerator uses localization time inside confinement intervals.
    Denominator uses localization time overall.

    tracks: list of dict/Series-like track objects
    confinements: list of tuples (i0, i1, ..., valid_flag) per track
                 i1 assumed INCLUSIVE. If yours is exclusive, change slice to slice(i0, i1).
    """
    r_all, dt_all = [], []
    r_conf_all, dt_conf_all = [], []

    # determine rmax if needed (from observed radii)
    if bins is None and rmax is None:
        rmax_tmp = 0.0
        for tr in tracks:
            x0, y0 = tr["roipos"]
            x = np.asarray(tr["x"])
            y = np.asarray(tr["y"])
            r = np.sqrt((x - x0)**2 + (y - y0)**2)
            if r.size:
                rmax_tmp = max(rmax_tmp, float(np.nanmax(r)))
        rmax = rmax_tmp

    if bins is None:
        bins = np.arange(0, rmax + dr, dr)

    for tr in tracks:
        if isinstance(tr["pc"], (list, np.ndarray)):
            x0, y0 = tr["roipos"]
            x = np.asarray(tr["x"])
            y = np.asarray(tr["y"])
            t = np.asarray(tr["tim"])

            r = np.sqrt((x - x0)**2 + (y - y0)**2)

            dts = np.diff(t)
            dt_last = np.median(dts) if dts.size else 0.0
            dt = np.append(dts, dt_last)

            r_all.append(r)
            dt_all.append(dt)

            for conf in tr.get("confinements", []):
                i0, i1 = int(conf[0]), int(conf[1])
                valid = True
                if len(conf) >= 4:
                    valid = bool(conf[3])
                if not valid:
                    continue

                sl = slice(i0, i1 + 1)  # inclusive end; change if needed
                r_conf_all.append(r[sl])
                dt_conf_all.append(dt[sl])

    r_all = np.concatenate(r_all) if r_all else np.array([])
    dt_all = np.concatenate(dt_all) if dt_all else np.array([])
    r_conf_all = np.concatenate(r_conf_all) if r_conf_all else np.array([])
    dt_conf_all = np.concatenate(dt_conf_all) if dt_conf_all else np.array([])

    Tobs, _ = np.histogram(r_all, bins=bins, weights=dt_all)
    Tconf, _ = np.histogram(r_conf_all, bins=bins, weights=dt_conf_all)

    P = np.full_like(Tobs, np.nan, dtype=float)
    mask = Tobs > 0
    P[mask] = Tconf[mask] / Tobs[mask]

    if min_time_per_bin > 0:
        P[Tobs < min_time_per_bin] = np.nan

    r_centers = 0.5 * (bins[:-1] + bins[1:])
    return r_centers, P, Tconf, Tobs, bins

In [ ]:
pack_window_t = 0.004
pack_window_locs = 25
min_t_streak = 0.010  # ms
min_streak_pts = 25
confinement_size_thresh = 70  # nm
pack_lim = 10**((3.2 - np.log10(confinement_size_thresh))/0.46)  # inverted formula to go from Pc to confinement size, from Renner et al 2017
folderidx = 0

folders = [os.path.join(parentdir, 'exampledata\\cav1\\random-dppe\\240214'),
           os.path.join(parentdir, 'exampledata\\cav1\\random-dppe\\240925'),
           os.path.join(parentdir, 'exampledata\\cav1\\site-dppe\\240209'),
           os.path.join(parentdir, 'exampledata\\cav1\\site-dppe\\240925'),
           os.path.join(parentdir, 'exampledata\\cav1\\random-sm\\240214'),
           os.path.join(parentdir, 'exampledata\\cav1\\random-sm\\240925'),
           os.path.join(parentdir, 'exampledata\\cav1\\site-sm\\240209'),
           os.path.join(parentdir, 'exampledata\\cav1\\site-sm\\240925')]

confocal_scan_params_all = [[2, True],
                            [2, True],
                            [2, True],
                            [2, False],
                            [2, False],
                            [2, True],
                            [2, True],
                            [2, True],
                            [2, False],
                            [2, False]]

folder = folders[folderidx]
confocal_scan_params = confocal_scan_params_all[folderidx]

# PACKING COEFFICIENT ANALYSIS
analysis = DiffusionAnalysis('packingcoeff')
for fig in figures:
    analysis.create_fig(fig, figsize=(3,3))
analysis.set_analysis_parameters(site_rad=analysis_parameter_vals[0], inclusion_rad=analysis_parameter_vals[1], circle_radii=analysis_parameter_vals[2],
                                blob_dist=analysis_parameter_vals[3], min_time=analysis_parameter_vals[4], split_len_thresh=analysis_parameter_vals[5],
                                max_time_lag=analysis_parameter_vals[6], max_dt=analysis_parameter_vals[7], fit_len_thresh=analysis_parameter_vals[8],
                                meanslidestd_thresh=analysis_parameter_vals[9], slidstd_interval=analysis_parameter_vals[10],
                                meanpos_thresh=analysis_parameter_vals[11], interval_meanpos=analysis_parameter_vals[12], interval_dist=analysis_parameter_vals[13])
# set confocal scan parameters, common to all data
analysis.set_confocal_params(conf_scan_params=confocal_scan_params)
# init confocal shift compensation, reading a function fitted on a certain pixel size (70 nm or 60 nm)
analysis.confocal_compensation_shift_init(fitfiles_folder='C:\\Users\\alvelidjonatan\\Documents\\Data\\etMINFLUX-lab\\beads-overlay\\confshift\\final_fits\\cav1')
analysis.add_data(folder, plotting=False)
analysis.fit_site_position()
analysis.filter_site_flagging()

analysis.track_data = packing_coeff_analysis_locs(analysis.track_data, n=pack_window_locs, pack_lim=pack_lim, plot=True)
analysis.track_data = label_confinement_mask(analysis.track_data, pack_window_t, pack_lim, min_dur_s=min_t_streak, min_pts=min_streak_pts, confinement_size_thresh=confinement_size_thresh*1e-3)
analysis.track_data = build_confinement_coords_from_mask(analysis.track_data)

# Plot tracks colored by thresholded confinements
plot_traj_conf(analysis.track_data, savefolder=folder, extend=1)
# Plot tracks with confinement period localizations highlighted, sized by confinement duration, overlaid on confocal scan
plot_confinements(analysis.track_data, savefolder=folder)

analysis.save_pickleddata(savesuffix='-pcana')

# Calculate parameters to plot
numconfinements = [len(val) for val in analysis.track_data['conf_events'] if isinstance(val, (list, np.ndarray))]
stateratios = np.array(analysis.track_data['state_ratios'].tolist())
stateratios = [val for val in stateratios if not np.isnan(val)]
statechangesperms = np.array(analysis.track_data['state_changes_per_ms'].tolist())
statechangesperms = [val for val in statechangesperms if not np.isnan(val)]
confevents = [track for track in analysis.track_data['conf_events'].tolist()]
confdurations = []
for t in confevents:
    if isinstance(t, (list, np.ndarray)):
        for dur in [e[6] for e in t]:
            confdurations.append(dur)
    
# Plot number of confinements per track
fig, ax = plt.subplots()
plt.hist(numconfinements, bins=np.arange(-0.5,8.5,1), density=True)
ax.set_xlabel("Number of confinements")
ax.set_ylabel("Density (arb.u.)")
plt.show()

# Plot confinement durations per track
fig, ax = plt.subplots()
plt.hist(confdurations, bins=np.arange(0, 0.6, 0.01), density=True)
ax.set_xlabel("Confinement duration (s)")
ax.set_ylabel("Density (arb.u.)")
plt.xlim([0,0.6])
plt.show()

# Plot state ratio and state changes/ms distributions
fig, ax = plt.subplots()
ax.hist(stateratios, bins=20, density=True)
ax.set_xlabel("Confinement time ratio")
ax.set_ylabel("Density (arb.u.)")
plt.show()

fig, ax = plt.subplots()
plt.hist(statechangesperms, bins=20, range=(0,0.5), density=True)
ax.set_xlabel("Confinement states per ms")
ax.set_ylabel("Density (arb.u.)")
plt.show()

# Plot radial profiles of confinement sojourns per localization and fraction of time confined
r, P, S, L = radial_profile_confinements(analysis.track_data, dr=0.04, rmax=1.0)
fig, ax = plt.subplots()
plt.plot(r, P, marker="o")
ax.set_xlabel("Radial distance from caveola center")
ax.set_ylabel("Confinement sojourns per localization")
ax.set_title("Radial confinement profile")
ax.set_ylim([0, 0.05])
plt.show()

r, P, Tconf, Tobs = radial_profile_conf_fraction(analysis.track_data, dr=0.04, rmax=1.0)
fig, ax = plt.subplots()
plt.plot(r, P, marker="o")
ax.set_xlabel("Radial distance from caveola center"); plt.ylabel("Fraction of time confined")
ax.set_ylim(0, 1)
plt.show()